In [ ]:
import torch
from diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion import StableDiffusionPipeline
from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
from PIL import Image
import os


In [ ]:
# 模型选择：使用了 Stable Diffusion v1.5，这是一个平衡了质量和速度的模型，适合移动版 GPU 运行
# 优化设置：
# 使用 FP16 精度减少显存占用
# 集成了 DPMSolver 调度器加速生成过程
# 支持 xFormers 加速（需要额外安装：pip install xformers）
# 禁用梯度计算减少显存使用
# 参数调整：
# 对于 RTX 5090 移动版，建议初始分辨率设置为 768x512 或 512x512
# 推理步数 25-30 步可以在质量和速度间取得平衡
# 引导尺度 7.5 左右可以很好地遵循提示词

# 如果想尝试更先进的模型，可以将model_id改为：
# "stabilityai/stable-diffusion-2-1"（更高质量）
# "runwayml/stable-diffusion-inpainting"（支持图像修复）
# 若遇到显存不足问题，可以：
# 降低图像分辨率
# 减少推理步数
# 添加pipe.enable_attention_slicing()启用注意力切片
# 对于更快的生成速度，可以尝试：
# 使用 "CompVis/stable-diffusion-v1-4" 等较小模型
# 减少推理步数至 20 步
def generate_image(prompt, output_path="output.png", model_id="runwayml/stable-diffusion-v1-5", 
                   width=512, height=512, num_inference_steps=25, guidance_scale=7.5):
    """
    使用Stable Diffusion生成图像
    
    参数:
        prompt: 文本提示词
        output_path: 输出图像路径
        model_id: 模型ID
        width: 生成图像宽度
        height: 生成图像高度
        num_inference_steps: 推理步数
        guidance_scale: 引导尺度，值越大越遵循提示词
    """
    # 检查CUDA是否可用
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用设备: {device}")
    
    if device != "cuda":
        print("警告: 未检测到CUDA，将使用CPU运行，速度会很慢")
        return
    
    # 加载模型并配置调度器
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,  # 使用FP16节省显存
        revision="fp16",
        safety_checker=None  # 关闭安全检查器（可选）
    )
    
    # 使用DPMSolver调度器加速生成
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    
    # 移动模型到GPU
    pipe = pipe.to(device)
    
    # 启用xFormers加速（如果安装了xFormers）
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("已启用xFormers加速")
    except:
        print("未安装xFormers，将使用默认注意力机制")
    
    # 生成图像
    with torch.no_grad():  # 禁用梯度计算节省显存
        image = pipe(
            prompt,
            width=width,
            height=height,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale
        ).images[0]
    
    # 保存图像
    image.save(output_path)
    print(f"图像已保存至: {output_path}")
    return image


In [ ]:

prompt = "a beautiful sunset over the mountains, detailed, vibrant colors, 8k resolution"

# 创建输出目录
os.makedirs("outputs_images", exist_ok=True)

# 生成图像
generate_image(
    prompt=prompt,
    output_path="outputs_images/sunset.png",
    width=768,  # RTX 5090移动版可以流畅处理这个分辨率
    height=512,
    num_inference_steps=30
)